In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgb
from sklearn.metrics import r2_score

/Applications/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df= pd.read_csv('./CSVs/Modelling.csv')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",None)

In [4]:
df.head()

,maximum_nights,extra_people,guests_included,beds,availability_365,accommodates,bedrooms,host_listings_count,host_for,number_of_reviews_ltm,minimum_nights,bathrooms,secure,cleaning_fee,room_type_private_room,host_acceptance_rate,elevator,white_goods,host_response_rate,security_deposit,government,review_scores_value,area_name_salamanca,area_name_san_blas,price
0,365,15.0,2,0.0,77,2,1.0,1.0,4010,14,1,1.0,0.0,1.0,1,3.0,1.0,1.0,4.0,0.0,1.0,10.0,0,0,70.0
1,40,8.0,1,1.0,200,1,1.0,2.0,3699,0,4,1.0,0.0,0.0,1,4.0,1.0,1.0,4.0,0.0,1.0,9.0,0,0,17.0
2,730,10.0,1,5.0,249,6,3.0,9.0,3703,0,15,2.0,0.0,1.0,0,3.0,1.0,1.0,4.0,1.0,1.0,9.0,0,0,50.0
3,730,10.0,2,1.0,364,3,0.0,1.0,3661,7,4,1.0,0.0,1.0,0,3.0,1.0,1.0,4.0,1.0,1.0,10.0,0,0,89.0
4,10,0.0,4,3.0,322,4,2.0,1.0,3661,16,3,1.5,0.0,0.0,0,4.0,1.0,1.0,4.0,1.0,0.0,10.0,0,0,115.0


In [5]:
df.dtypes

maximum_nights              int64
extra_people              float64
guests_included             int64
beds                      float64
availability_365            int64
accommodates                int64
bedrooms                  float64
host_listings_count       float64
host_for                    int64
number_of_reviews_ltm       int64
minimum_nights              int64
bathrooms                 float64
secure                    float64
cleaning_fee              float64
room_type_private_room      int64
host_acceptance_rate      float64
elevator                  float64
white_goods               float64
host_response_rate        float64
security_deposit          float64
government                float64
review_scores_value       float64
area_name_salamanca         int64
area_name_san_blas          int64
price                     float64
dtype: object

# Train Test Split

In [6]:
splitter = train_test_split
"-----------------------"

df_train, df_test = splitter(df, test_size = 0.3, random_state = 42)
print("Dataset shape: {shape}".format(shape = df_train.shape))
print("Dataset shape: {shape}".format(shape = df_test.shape))

Dataset shape: (15283, 25)
Dataset shape: (6550, 25)


In [7]:
target = 'price'
X_train= df_train.drop('price',axis=1)
y_train = df_train[target]

X_test= df_test.drop('price',axis=1)
y_test = df_test[target]

# Standardizing and Normalizing

In [8]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)
X_train = pd.DataFrame(X_train_sc, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_test_sc, index=X_test.index, columns=X_test.columns)

In [9]:
scaler = MinMaxScaler()
X_trains = scaler.fit_transform(X_train_sc)
X_tests = scaler.fit_transform(X_test_sc)
X_train = pd.DataFrame(X_trains, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_tests, index=X_test.index, columns=X_test.columns)

In [10]:
X_train.head()

,maximum_nights,extra_people,guests_included,beds,availability_365,accommodates,bedrooms,host_listings_count,host_for,number_of_reviews_ltm,minimum_nights,bathrooms,secure,cleaning_fee,room_type_private_room,host_acceptance_rate,elevator,white_goods,host_response_rate,security_deposit,government,review_scores_value,area_name_salamanca,area_name_san_blas
3693,1.011600e-05,0.000000,0.000000,0.02,0.000000,0.052632,0.111111,0.001898,0.331536,0.000000,0.003559,0.090909,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.875,0.0,0.0
6385,1.011600e-05,0.000000,0.000000,0.02,0.715068,0.000000,0.111111,0.009488,0.478314,0.039877,0.024021,0.090909,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.875,0.0,0.0
4029,1.260000e-07,0.000000,0.000000,0.04,0.000000,0.157895,0.111111,0.001898,0.635874,0.000000,0.003559,0.090909,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.875,1.0,0.0
7220,8.010000e-07,0.055762,0.066667,0.04,0.873973,0.263158,0.222222,0.091082,0.520706,0.079755,0.001779,0.090909,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.000,0.0,0.0
3995,1.800000e-07,0.037175,0.000000,0.02,0.158904,0.052632,0.111111,0.001898,0.323940,0.015337,0.001779,0.090909,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.875,0.0,0.0


In [11]:
X_train= df_train.drop('price', axis=1)
y_train = df_train['price']
X_test= df_test.drop('price', axis=1)
y_test= df_test['price']

In [12]:
def mean_absolute_percentage_error(y_true,y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return (np.sum(np.abs(y_true-y_pred)/y_true)/len(y_true))*100

# LGBM Model

In [13]:
#get orientative parameters
if __name__ == "__main__":
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_test, label=y_test)

    params = {
        "objective": "regression",
        "metric": "mape",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }

    model = lgb.train(
        params, dtrain, valid_sets=[dtrain, dval], verbose_eval=100, early_stopping_rounds=100,
    )

    prediction = np.rint(model.predict(X_test, num_iteration=model.best_iteration))
    accuracy = r2_score(y_test,prediction)

    best_params = model.params
    print("Best params:", best_params)
    print("  Accuracy = {}".format(accuracy))
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))

/Applications/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 0.846979	valid_1's mape: 0.934142
Early stopping, best iteration is:
[52]	training's mape: 0.834264	valid_1's mape: 0.889923


tune_feature_fraction, val_score: 0.889923:  14%|#4        | 1/7 [00:01<00:05,  1.01it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 0.806142	valid_1's mape: 0.902303
Early stopping, best iteration is:
[53]	training's mape: 0.804952	valid_1's mape: 0.864298


tune_feature_fraction, val_score: 0.889923:  29%|##8       | 2/7 [00:03<00:06,  1.36s/it]

...

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 1.08155	valid_1's mape: 1.08421
[200]	training's mape: 1.08793	valid_1's mape: 1.08275
Early stopping, best iteration is:
[130]	training's mape: 1.0505	valid_1's mape: 1.04827


tune_min_child_samples, val_score: 1.070724:  80%|########  | 4/5 [00:02<00:00,  2.20it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 1.20183	valid_1's mape: 1.18244
[200]	training's mape: 1.16813	valid_1's mape: 1.14133
Early stopping, best iteration is:
[166]	training's mape: 1.14676	valid_1's mape: 1.1223


tune_min_child_samples, val_score: 1.122297: 100%|##########| 5/5 [00:02<00:00,  2.03it/s]


Best params: {'objective': 'regression', 'metric': 'mape', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 8.37345418497093, 'lambda_l2': 0.0054026995561138776, 'num_leaves': 4, 'feature_fraction': 0.4, 'bagging_fraction': 0.9274928940073707, 'bagging_freq': 6, 'min_child_samples': 100}
  Accuracy = 0.638093236954413
  Params: 
    objective: regression
    metric: mape
    verbosity: -1
    boosting_type: gbdt
    lambda_l1: 8.37345418497093
    lambda_l2: 0.0054026995561138776
    num_leaves: 4
    feature_fraction: 0.4
    bagging_fraction: 0.9274928940073707
    bagging_freq: 6
    min_child_samples: 100


In [14]:
params= {}
params['objective']='regression'
params['metric']='mape'
params['verbosity']= -1
params['boosting_type']= 'gbdt'
params['lambda_l1']= 1.1016383599645503e-08
params['lambda_l2']= 0.009904089144349917
params['num_leaves']= 191
params['feature_fraction']= 0.4
params['bagging_fraction']= 0.45136537560731055
params['bagging_freq']= 4
params['min_child_samples']= 100

In [15]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

In [16]:
#optimizing
clf=lgb.train(params,train_data,valid_sets=test_data,early_stopping_rounds=100)

/Applications/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's mape: 2.09106
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 1.9948
[3]	valid_0's mape: 1.91248
[4]	valid_0's mape: 1.85485
[5]	valid_0's mape: 1.77694
[6]	valid_0's mape: 1.73121
[7]	valid_0's mape: 1.66566
[8]	valid_0's mape: 1.61124
[9]	valid_0's mape: 1.55291
[10]	valid_0's mape: 1.50472
[11]	valid_0's mape: 1.46767
[12]	valid_0's mape: 1.44067
[13]	valid_0's mape: 1.40552
[14]	valid_0's mape: 1.37536
[15]	valid_0's mape: 1.34953
[16]	valid_0's mape: 1.31917
[17]	valid_0's mape: 1.29545
[18]	valid_0's mape: 1.26445
[19]	valid_0's mape: 1.24762
[20]	valid_0's mape: 1.22065
[21]	valid_0's mape: 1.20713
[22]	valid_0's mape: 1.18353
[23]	valid_0's mape: 1.16893
[24]	valid_0's mape: 1.14921
[25]	valid_0's mape: 1.13006
[26]	valid_0's mape: 1.10895
[27]	valid_0's mape: 1.09958
[28]	valid_0's mape: 1.09061
[29]	valid_0's mape: 1.07856
[30]	valid_0's mape: 1.06823
[31]	valid_0's mape: 1.07276
[32]	valid_0's mape: 1.06968
[33]	valid_0's mape:

tune_feature_fraction, val_score: 1.062901:  14%|#4        | 1/7 [00:03<00:18,  3.07s/it]

[1]	valid_0's mape: 2.08553
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 1.98849
[3]	valid_0's mape: 1.89882
[4]	valid_0's mape: 1.83924
[5]	valid_0's mape: 1.76076
[6]	valid_0's mape: 1.70831
[7]	valid_0's mape: 1.6419
[8]	valid_0's mape: 1.57004
[9]	valid_0's mape: 1.51569
[10]	valid_0's mape: 1.46881
[11]	valid_0's mape: 1.42886
[12]	valid_0's mape: 1.40481
[13]	valid_0's mape: 1.35575
[14]	valid_0's mape: 1.3285
[15]	valid_0's mape: 1.2936
[16]	valid_0's mape: 1.26567
[17]	valid_0's mape: 1.24663
[18]	valid_0's mape: 1.21336
[19]	valid_0's mape: 1.18918
[20]	valid_0's mape: 1.18047
[21]	valid_0's mape: 1.16502
[22]	valid_0's mape: 1.14161
[23]	valid_0's mape: 1.13408
[24]	valid_0's mape: 1.11369
[25]	valid_0's mape: 1.09151
[26]	valid_0's mape: 1.07679
[27]	valid_0's mape: 1.06616
[28]	valid_0's mape: 1.05417
[29]	valid_0's mape: 1.04826
[30]	valid_0's mape: 1.04196
[31]	valid_0's mape: 1.04236
[32]	valid_0's mape: 1.03756
[33]	valid_0's mape: 1

tune_feature_fraction, val_score: 1.062901:  29%|##8       | 2/7 [00:07<00:17,  3.41s/it]

...

tune_min_child_samples, val_score: 1.387207:  60%|######    | 3/5 [00:07<00:05,  2.62s/it]

[1]	valid_0's mape: 2.1215
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 2.03995
[3]	valid_0's mape: 1.98129
[4]	valid_0's mape: 1.93116
[5]	valid_0's mape: 1.86085
[6]	valid_0's mape: 1.83768
[7]	valid_0's mape: 1.77228
[8]	valid_0's mape: 1.71452
[9]	valid_0's mape: 1.66288
[10]	valid_0's mape: 1.62383
[11]	valid_0's mape: 1.59604
[12]	valid_0's mape: 1.57035
[13]	valid_0's mape: 1.54343
[14]	valid_0's mape: 1.52772
[15]	valid_0's mape: 1.48949
[16]	valid_0's mape: 1.47297
[17]	valid_0's mape: 1.45425
[18]	valid_0's mape: 1.43579
[19]	valid_0's mape: 1.43169
[20]	valid_0's mape: 1.4274
[21]	valid_0's mape: 1.42752
[22]	valid_0's mape: 1.41363
[23]	valid_0's mape: 1.41089
[24]	valid_0's mape: 1.40214
[25]	valid_0's mape: 1.38721
[26]	valid_0's mape: 1.39149
[27]	valid_0's mape: 1.40623
[28]	valid_0's mape: 1.4073
[29]	valid_0's mape: 1.40486
[30]	valid_0's mape: 1.40391
[31]	valid_0's mape: 1.41014
[32]	valid_0's mape: 1.40379
[33]	valid_0's mape: 1

tune_min_child_samples, val_score: 1.387207:  80%|########  | 4/5 [00:09<00:02,  2.44s/it]

[1]	valid_0's mape: 2.1215
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 2.03995
[3]	valid_0's mape: 1.98129
[4]	valid_0's mape: 1.93116
[5]	valid_0's mape: 1.86085
[6]	valid_0's mape: 1.83768
[7]	valid_0's mape: 1.77228
[8]	valid_0's mape: 1.71452
[9]	valid_0's mape: 1.66288
[10]	valid_0's mape: 1.62383
[11]	valid_0's mape: 1.59604
[12]	valid_0's mape: 1.57035
[13]	valid_0's mape: 1.54343
[14]	valid_0's mape: 1.52772
[15]	valid_0's mape: 1.48949
[16]	valid_0's mape: 1.47297
[17]	valid_0's mape: 1.45425
[18]	valid_0's mape: 1.43579
[19]	valid_0's mape: 1.43169
[20]	valid_0's mape: 1.4274
[21]	valid_0's mape: 1.42752
[22]	valid_0's mape: 1.41363
[23]	valid_0's mape: 1.41089
[24]	valid_0's mape: 1.40214
[25]	valid_0's mape: 1.38721
[26]	valid_0's mape: 1.39149
[27]	valid_0's mape: 1.40623
[28]	valid_0's mape: 1.4073
[29]	valid_0's mape: 1.40486
[30]	valid_0's mape: 1.40391
[31]	valid_0's mape: 1.41014
[32]	valid_0's mape: 1.40379
[33]	valid_0's mape: 1

tune_min_child_samples, val_score: 1.387207: 100%|##########| 5/5 [00:11<00:00,  2.33s/it]


In [17]:
pred_train = clf.predict(X_train, num_iteration=clf.best_iteration)
pred_test= clf.predict(X_test, num_iteration=clf.best_iteration)

In [18]:
print('TRAIN MAPE:', mean_absolute_percentage_error(y_train,pred_train))
print('TEST MAPE:', mean_absolute_percentage_error(y_test,pred_test))

TRAIN MAPE: 138.10908578897838
TEST MAPE: 138.72066810673894


## Nested Validation

In [19]:
splitter = train_test_split
"-----------------------"

df_model, df_val = splitter(df, test_size = 0.3, random_state = 42)
print("Dataset shape: {shape}".format(shape = df_model.shape))
print("Dataset shape: {shape}".format(shape = df_val.shape))

Dataset shape: (15283, 25)
Dataset shape: (6550, 25)


In [20]:
df_train, df_test = splitter(df_model, test_size = 0.3, random_state = 42)
print("Dataset shape: {shape}".format(shape = df_train.shape))
print("Dataset shape: {shape}".format(shape = df_val.shape))

Dataset shape: (10698, 25)
Dataset shape: (6550, 25)


In [21]:
target = 'price'
X_train= df_train.drop('price',axis=1)
y_train = df_train[target]

X_test= df_test.drop('price',axis=1)
y_test = df_test[target]

X_val= df_val.drop('price',axis=1)
y_val = df_val[target]

In [22]:
# Standardizing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)
X_val_sc = scaler.fit_transform(X_val)
X_train = pd.DataFrame(X_train_sc, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_test_sc, index=X_test.index, columns=X_test.columns)
X_val = pd.DataFrame(X_val_sc, index=X_val.index, columns=X_val.columns)

In [23]:
# Normalizing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_trains = scaler.fit_transform(X_train_sc)
X_tests = scaler.fit_transform(X_test_sc)
X_vals = scaler.fit_transform(X_val_sc)
X_train = pd.DataFrame(X_trains, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_tests, index=X_test.index, columns=X_test.columns)
X_val = pd.DataFrame(X_vals, index=X_val.index, columns=X_val.columns)

In [25]:
if __name__ == "__main__":
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_test, label=y_test)

    params = {
        "objective": "regression",
        "metric": "mape",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }

    model = lgb.train(
        params, dtrain, valid_sets=[dtrain, dval], verbose_eval=100, early_stopping_rounds=100,
    )

    prediction = np.rint(model.predict(X_test, num_iteration=model.best_iteration))
    accuracy = r2_score(y_test,prediction)

    best_params = model.params
    print("Best params:", best_params)
    print("  Accuracy = {}".format(accuracy))
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))

/Applications/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 0.883239	valid_1's mape: 1.42347
Early stopping, best iteration is:
[49]	training's mape: 0.859104	valid_1's mape: 1.22438


tune_feature_fraction, val_score: 1.224382:  14%|#4        | 1/7 [00:00<00:02,  2.45it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 0.864653	valid_1's mape: 1.42867
Early stopping, best iteration is:
[35]	training's mape: 0.869521	valid_1's mape: 1.28532


tune_feature_fraction, val_score: 1.285318:  29%|##8       | 2/7 [00:01<00:02,  2.19it/s]

...

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 0.781592	valid_1's mape: 1.66875
Early stopping, best iteration is:
[29]	training's mape: 0.823554	valid_1's mape: 1.25215


tune_min_child_samples, val_score: 1.736998:  80%|########  | 4/5 [00:04<00:01,  1.11s/it]

Training until validation scores don't improve for 100 rounds
[100]	training's mape: 1.00847	valid_1's mape: 1.78451
Early stopping, best iteration is:
[25]	training's mape: 1.12369	valid_1's mape: 1.56292


tune_min_child_samples, val_score: 1.736998: 100%|##########| 5/5 [00:05<00:00,  1.20s/it]


Best params: {'objective': 'regression', 'metric': 'mape', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 5.532206663012917e-05, 'lambda_l2': 2.565356651531435e-08, 'num_leaves': 216, 'feature_fraction': 0.6, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 10}
  Accuracy = 0.5515595615175944
  Params: 
    objective: regression
    metric: mape
    verbosity: -1
    boosting_type: gbdt
    lambda_l1: 5.532206663012917e-05
    lambda_l2: 2.565356651531435e-08
    num_leaves: 216
    feature_fraction: 0.6
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 10


In [26]:
params= {}
params['objective']='regression'
params['metric']='mape'
params['verbosity']= -1
params['boosting_type']= 'gbdt'
params['lambda_l1']= 0.0053300036379407860
params['lambda_l2']= 0.03145917157835856
params['num_leaves']= 140
params['feature_fraction']= 0.4
params['bagging_fraction']= 0.43436513528744136
params['bagging_freq']= 6
params['min_child_samples']= 100

In [27]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

In [28]:
clf=lgb.train(params,train_data,valid_sets=test_data,early_stopping_rounds=100)

/Applications/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's mape: 2.18878
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 2.28548
[3]	valid_0's mape: 2.20373
[4]	valid_0's mape: 2.15636
[5]	valid_0's mape: 2.09607
[6]	valid_0's mape: 2.09906
[7]	valid_0's mape: 2.02245
[8]	valid_0's mape: 1.97834
[9]	valid_0's mape: 1.91733
[10]	valid_0's mape: 1.87065
[11]	valid_0's mape: 1.83093
[12]	valid_0's mape: 1.80519
[13]	valid_0's mape: 1.76693
[14]	valid_0's mape: 1.74691
[15]	valid_0's mape: 1.76103
[16]	valid_0's mape: 1.74691
[17]	valid_0's mape: 1.71887
[18]	valid_0's mape: 1.68059
[19]	valid_0's mape: 1.64859
[20]	valid_0's mape: 1.61664
[21]	valid_0's mape: 1.61259
[22]	valid_0's mape: 1.58379
[23]	valid_0's mape: 1.57495
[24]	valid_0's mape: 1.54748
[25]	valid_0's mape: 1.53242
[26]	valid_0's mape: 1.54099
[27]	valid_0's mape: 1.60285
[28]	valid_0's mape: 1.59735
[29]	valid_0's mape: 1.58439
[30]	valid_0's mape: 1.57211
[31]	valid_0's mape: 1.62005
[32]	valid_0's mape: 1.60217
[33]	valid_0's mape

tune_feature_fraction, val_score: 1.532423:  14%|#4        | 1/7 [00:04<00:26,  4.41s/it]

[1]	valid_0's mape: 2.19252
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 2.28793
[3]	valid_0's mape: 2.19673
[4]	valid_0's mape: 2.14713
[5]	valid_0's mape: 2.07922
[6]	valid_0's mape: 2.08491
[7]	valid_0's mape: 2.01544
[8]	valid_0's mape: 1.95813
[9]	valid_0's mape: 1.97342
[10]	valid_0's mape: 1.92519
[11]	valid_0's mape: 1.88068
[12]	valid_0's mape: 1.85794
[13]	valid_0's mape: 1.83237
[14]	valid_0's mape: 1.81107
[15]	valid_0's mape: 1.78027
[16]	valid_0's mape: 1.74542
[17]	valid_0's mape: 1.71473
[18]	valid_0's mape: 1.67731
[19]	valid_0's mape: 1.66208
[20]	valid_0's mape: 1.63461
[21]	valid_0's mape: 1.62621
[22]	valid_0's mape: 1.60563
[23]	valid_0's mape: 1.59862
[24]	valid_0's mape: 1.57189
[25]	valid_0's mape: 1.55452
[26]	valid_0's mape: 1.5526
[27]	valid_0's mape: 1.60827
[28]	valid_0's mape: 1.60003
[29]	valid_0's mape: 1.62407
[30]	valid_0's mape: 1.59633
[31]	valid_0's mape: 1.64224
[32]	valid_0's mape: 1.62339
[33]	valid_0's mape:

tune_feature_fraction, val_score: 1.552595:  29%|##8       | 2/7 [00:09<00:22,  4.52s/it]

...

tune_min_child_samples, val_score: 1.816923:  60%|######    | 3/5 [00:05<00:03,  1.74s/it]

[1]	valid_0's mape: 2.2212
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 2.14044
[3]	valid_0's mape: 2.10507
[4]	valid_0's mape: 2.08418
[5]	valid_0's mape: 2.03987
[6]	valid_0's mape: 2.0175
[7]	valid_0's mape: 1.98203
[8]	valid_0's mape: 1.92569
[9]	valid_0's mape: 2.00129
[10]	valid_0's mape: 1.98137
[11]	valid_0's mape: 1.98937
[12]	valid_0's mape: 1.98923
[13]	valid_0's mape: 1.97871
[14]	valid_0's mape: 1.97042
[15]	valid_0's mape: 1.92858
[16]	valid_0's mape: 1.92577
[17]	valid_0's mape: 1.90459
[18]	valid_0's mape: 1.88332
[19]	valid_0's mape: 1.86207
[20]	valid_0's mape: 1.83693
[21]	valid_0's mape: 1.82829
[22]	valid_0's mape: 1.84502
[23]	valid_0's mape: 1.84727
[24]	valid_0's mape: 1.83806
[25]	valid_0's mape: 1.82933
[26]	valid_0's mape: 1.82898
[27]	valid_0's mape: 1.83469
[28]	valid_0's mape: 1.85407
[29]	valid_0's mape: 1.84137
[30]	valid_0's mape: 1.81692
[31]	valid_0's mape: 1.95216
[32]	valid_0's mape: 1.93599
[33]	valid_0's mape: 

tune_min_child_samples, val_score: 1.816923:  80%|########  | 4/5 [00:06<00:01,  1.74s/it]

[1]	valid_0's mape: 2.2212
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mape: 2.14044
[3]	valid_0's mape: 2.10507
[4]	valid_0's mape: 2.08418
[5]	valid_0's mape: 2.03987
[6]	valid_0's mape: 2.0175
[7]	valid_0's mape: 1.98203
[8]	valid_0's mape: 1.92569
[9]	valid_0's mape: 2.00129
[10]	valid_0's mape: 1.98137
[11]	valid_0's mape: 1.98937
[12]	valid_0's mape: 1.98923
[13]	valid_0's mape: 1.97871
[14]	valid_0's mape: 1.97042
[15]	valid_0's mape: 1.92858
[16]	valid_0's mape: 1.92577
[17]	valid_0's mape: 1.90459
[18]	valid_0's mape: 1.88332
[19]	valid_0's mape: 1.86207
[20]	valid_0's mape: 1.83693
[21]	valid_0's mape: 1.82829
[22]	valid_0's mape: 1.84502
[23]	valid_0's mape: 1.84727
[24]	valid_0's mape: 1.83806
[25]	valid_0's mape: 1.82933
[26]	valid_0's mape: 1.82898
[27]	valid_0's mape: 1.83469
[28]	valid_0's mape: 1.85407
[29]	valid_0's mape: 1.84137
[30]	valid_0's mape: 1.81692
[31]	valid_0's mape: 1.95216
[32]	valid_0's mape: 1.93599
[33]	valid_0's mape: 

tune_min_child_samples, val_score: 1.816923: 100%|##########| 5/5 [00:10<00:00,  2.03s/it]


In [29]:
pred_train = clf.predict(X_train, num_iteration=clf.best_iteration)
pred_test= clf.predict(X_test, num_iteration=clf.best_iteration)
pred_val=clf.predict(X_val, num_iteration=clf.best_iteration)

In [30]:
print('TRAIN MAPE:', mean_absolute_percentage_error(y_train,pred_train))
print('TEST MAPE:', mean_absolute_percentage_error(y_test,pred_test))
print('VALIDATION MAPE:', mean_absolute_percentage_error(y_val,pred_val))

TRAIN MAPE: 162.2812706882525
TEST MAPE: 181.69233681147713
VALIDATION MAPE: 161.10287417871342


We discard using nested validation, because we don't have that many observations.

Camila Barbagallo, Ryan Daher, Paula García, Rocío González Lantero